# Initialize Everything

In [21]:
from pymongo import MongoClient
import pandas as pd

## Read MongoDB

In [22]:
client = MongoClient()
db = client["test"]
collection = db["matchapps"]
df = pd.json_normalize(collection.find())

## Reorganize and clean Data

In [23]:
# df.drop("__v", axis=1, inplace=True)

column_order = [
    "_id",
    "metadata.scouterName",
    "metadata.robotTeam",
    "metadata.robotPosition",
    "leftStartingZone",
    "climb",
    "autoAmpNotes",
    "teleAmpNotes",
    "trapNotes",
    "highNotes",
    "autoSpeakerNotes.near",
    "autoSpeakerNotes.mid",
    "autoSpeakerNotes.far",
    "teleNonAmpedSpeakerNotes.near",
    "teleNonAmpedSpeakerNotes.mid",
    "teleNonAmpedSpeakerNotes.far",
    "teleAmpedSpeakerNotes.near",
    "teleAmpedSpeakerNotes.mid",
    "teleAmpedSpeakerNotes.far"
]

df = df[column_order]

## Create Output DF

In [24]:
output_df = pd.DataFrame()

# Helpful Function and Variable Initializations

## Complete List of all Teams in DF

In [25]:
team_list = df["metadata.robotTeam"].unique().tolist()

# Analyze Data

## Tele Speaker Notes Average (near, mid, far)

In [26]:
output_df["Avg Tele Speaker Notes Near"] = df.groupby("metadata.robotTeam")["teleNonAmpedSpeakerNotes.near"].mean()
output_df["Avg Tele Speaker Notes Mid"] = df.groupby("metadata.robotTeam")["teleNonAmpedSpeakerNotes.mid"].mean()
output_df["Avg Tele Speaker Notes Far"] = df.groupby("metadata.robotTeam")["teleNonAmpedSpeakerNotes.far"].mean()

## Tele Amped Speaker Notes Average (near, mid, far)

In [27]:
output_df["Avg Amped Speaker Notes Near"] = df.groupby("metadata.robotTeam")["teleAmpedSpeakerNotes.near"].mean()
output_df["Avg Amped Speaker Notes Mid"] = df.groupby("metadata.robotTeam")["teleAmpedSpeakerNotes.mid"].mean()
output_df["Avg Amped Speaker Notes Far"] = df.groupby("metadata.robotTeam")["teleAmpedSpeakerNotes.far"].mean()

## Tele Amp Notes Average

In [28]:
output_df["Avg Tele Amp Notes"] = df.groupby("metadata.robotTeam")["teleAmpNotes"].mean()

## Auto Speaker Notes Average (near, mid, far)

In [29]:
output_df["Avg Auto Speaker Notes Near"] = df.groupby("metadata.robotTeam")["autoSpeakerNotes.near"].mean()
output_df["Avg Auto Speaker Notes Mid"] = df.groupby("metadata.robotTeam")["autoSpeakerNotes.mid"].mean()
output_df["Avg Auto Speaker Notes Far"] = df.groupby("metadata.robotTeam")["autoSpeakerNotes.far"].mean()

## Auto Amp Notes Average

In [30]:
output_df["Avg Auto Amp Notes"] = df.groupby("metadata.robotTeam")["autoAmpNotes"].mean()

## Trap Notes

In [31]:
output_df["Avg Trap Notes"] = df.groupby("metadata.robotTeam")["trapNotes"].mean()

## Percent of Successful Climbs

In [35]:
output_df["Percent of Successful Climbs"] = None
for team in team_list:
    climb_list = df.loc[df["metadata.robotTeam"] == team, "climb"].tolist()
    total_climbs = 0
    successful_climbs = 0
    for climb_type in climb_list:
        if climb_type == "failed":
            total_climbs += 1
        elif climb_type == "amp" or climb_type == "source" or climb_type == "center":
            total_climbs += 1
            successful_climbs += 1
    if total_climbs == 0:
        output_df.at[team, "Percent of Successful Climbs"] = 0
    else:
        output_df.at[team, "Percent of Successful Climbs"] = successful_climbs / total_climbs

## Percent of Climbs that Harmonized

## Percent of Times Robot Left Starting Zone During Autos

In [36]:
output_df["Percent of Times Robot Left Starting Zone During Autos"] = df.groupby("metadata.robotTeam")["leftStartingZone"].mean()

# Send Data

In [37]:
output_df
#df.to_json("../server/static/analyzed_data.json", orient='records', default_handler=str)

,Avg Tele Speaker Notes Near,Avg Tele Speaker Notes Mid,Avg Tele Speaker Notes Far,Avg Amped Speaker Notes Near,Avg Amped Speaker Notes Mid,Avg Amped Speaker Notes Far,Avg Tele Amp Notes,Avg Auto Speaker Notes Near,Avg Auto Speaker Notes Mid,Avg Auto Speaker Notes Far,Avg Auto Amp Notes,Avg Trap Notes,Percent of Successful Climbs,Percent of Times Robot Left Starting Zone During Autos
metadata.robotTeam,,,,,,,,,,,,,,
47,0.500000,1.000000,2.000000,2.000000,2.000000,2.000,3.000000,3.000000,2.000000,2.500000,1.000000,0.500000,0,0.000000
83,3.000000,0.666667,3.000000,2.000000,2.000000,2.000,3.333333,1.000000,1.333333,1.333333,3.333333,0.666667,0.666667,0.000000
105,2.142857,2.142857,3.142857,1.857143,1.571429,1.000,2.000000,1.571429,1.857143,2.285714,2.571429,0.428571,0.5,0.285714
157,4.000000,4.000000,3.000000,4.000000,3.000000,4.000,0.000000,3.000000,3.000000,3.000000,1.000000,1.000000,0,1.000000
173,2.000000,1.000000,3.000000,0.000000,2.000000,3.000,2.000000,3.000000,2.000000,2.000000,4.000000,1.000000,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9451,2.125000,1.250000,2.625000,1.875000,1.125000,1.375,2.375000,1.125000,2.625000,2.000000,1.875000,0.375000,0.8,0.500000
9530,2.000000,2.666667,1.666667,3.000000,3.000000,2.500,1.000000,2.666667,1.833333,1.666667,1.833333,0.333333,0.833333,0.333333
9678,0.500000,2.000000,3.000000,1.500000,1.500000,3.000,2.500000,1.500000,2.500000,3.000000,1.000000,0.500000,1.0,0.500000
